In [33]:
###-----------------
### Import libraries
###-----------------
from pathlib import Path  # Import Path for file system path operations and management
import numpy as np  # Import NumPy for numerical computations and array operations
import pandas as pd  # Import Pandas for data manipulation and analysis with DataFrames
import matplotlib.pyplot as plt  # Import Matplotlib for creating static, interactive visualizations
import seaborn as sns  # Import Seaborn for statistical data visualization built on Matplotlib

from sklearn.model_selection import  train_test_split # Import function to split dataset into training and testing subsets
from sklearn.metrics import * # Import function to calculate various metric

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import os


In [113]:
###----------------------
### Some basic parameters
###----------------------

inpDir = Path('..') / '..' / 'input'
outDir = Path('..') / 'output'
modelDir = Path('..') / 'models'
subDir = 'fifa_2019'

RANDOM_STATE = 24 # for initialization ----- REMEMBER: to remove at the time of promotion to production
np.random.seed(RANDOM_STATE)
rng = np.random.default_rng(seed = RANDOM_STATE) # Set Random Seed for reproducible  results

EPOCHS = 100 # number of epochs
BATCH_SIZE = 32
ALPHA = 0.001 # learning rate
TEST_SIZE = 0.2
# TRAIN_SIZE=int(18*BATCH_SIZE)


WEIGHT_DECAY = 0.001
PATENCE = 20
LR_FACTOR = 0.1
LR_PATIENCE = 5
MIN_LR = 1e-6

# parameters for Matplotlib
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (15, 6),
          'axes.labelsize': 'x-large',
          'axes.titlesize':'x-large',
          'xtick.labelsize':'x-large',
          'ytick.labelsize':'x-large'
         }

plt.rcParams.update(params)

CMAP = plt.cm.coolwarm
plt.style.use('seaborn-v0_8-darkgrid') # plt.style.use('ggplot')

In [ ]:
data_dir = r"D:\dnn_input\flower_photos"
IMAGE_SIZE = (64,64)
BATCH_SIZE = 32
validation_split_ratio = 0.2

## Use this when you have train and test as a seperate folder with classes

In [115]:
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   rotation_range = 20,
                                   width_shift_range=0.1,          # Randomly shift images horizontally
                                   height_shift_range=0.1,         # Randomly shift images vertically
                                   shear_range=0.1,                # Apply random shearing transformations
                                   zoom_range=0.1,                 # Apply random zoom
                                   horizontal_flip=True,           # Randomly flip images horizontally
                                   fill_mode='nearest'             # Strategy for filling in new pixels after transformation
)

test_datagen = ImageDataGenerator(rescale =1.0/255)

In [30]:
train_data = train_datagen.flow_from_directory(directory=train_dir,
                                               target_size=IMAGE_SIZE,
                                               batch_size = BATCH_SIZE,
                                               class_mode = 'categorical')
test_data = test_datagen.flow_from_directory(directory=test_dir,
                                               target_size=IMAGE_SIZE,
                                               batch_size = BATCH_SIZE,
                                               class_mode = 'categorical')

NameError: name 'train_dir' is not defined

## When one directory 

In [116]:
train_data = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=validation_split_ratio,
    subset = 'training',
    seed =123,
    image_size = IMAGE_SIZE,
    batch_size = BATCH_SIZE 
)

validation_data = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=validation_split_ratio,
    subset="validation",
    seed = 123,
    image_size = IMAGE_SIZE,
    batch_size = BATCH_SIZE
)

Found 3670 files belonging to 5 classes.
Using 2936 files for training.
Found 3670 files belonging to 5 classes.
Using 734 files for validation.


In [117]:
train_data.class_names

['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']

In [118]:
for batch,(data,labels) in enumerate(train_data):
    print(batch,data.shape,labels)

0 (32, 64, 64, 3) tf.Tensor([2 1 4 3 1 2 1 2 4 1 4 4 3 4 1 2 0 4 1 1 1 4 3 2 3 1 4 2 2 3 4 3], shape=(32,), dtype=int32)
1 (32, 64, 64, 3) tf.Tensor([0 2 4 4 4 1 1 1 1 1 3 0 1 2 4 0 3 1 0 0 4 4 4 2 4 1 1 0 4 3 2 0], shape=(32,), dtype=int32)
2 (32, 64, 64, 3) tf.Tensor([4 0 2 4 1 2 2 4 0 0 1 3 4 3 4 4 4 3 0 3 4 3 0 0 1 2 0 1 1 4 1 1], shape=(32,), dtype=int32)
3 (32, 64, 64, 3) tf.Tensor([2 3 4 4 3 4 4 2 0 1 0 1 0 2 2 4 1 4 2 3 3 2 1 2 1 3 2 4 1 4 1 4], shape=(32,), dtype=int32)
4 (32, 64, 64, 3) tf.Tensor([4 1 1 2 4 4 0 4 3 0 2 3 2 1 1 1 1 3 4 4 3 3 2 3 1 3 4 2 4 2 1 2], shape=(32,), dtype=int32)
5 (32, 64, 64, 3) tf.Tensor([4 0 1 1 4 3 2 4 1 1 2 4 4 4 1 3 4 4 3 3 1 1 3 3 3 4 4 2 1 2 0 0], shape=(32,), dtype=int32)
6 (32, 64, 64, 3) tf.Tensor([3 2 0 3 1 1 4 1 3 4 1 3 3 1 3 4 2 0 1 3 2 3 3 1 1 3 1 0 1 1 2 1], shape=(32,), dtype=int32)
7 (32, 64, 64, 3) tf.Tensor([1 4 0 2 0 1 1 1 3 1 4 4 3 3 3 2 1 2 4 1 0 4 0 1 4 4 4 1 3 4 2 1], shape=(32,), dtype=int32)
8 (32, 64, 64, 3) tf.Tensor([1 1

In [119]:
## First 10 batches
small_subset = train_data.take(1)
for data, labels in small_subset:
   for cnt,(features,label) in enumerate(zip(data,labels)):
      print(cnt,features,label)


0 tf.Tensor(
[[[ 90.40625  126.40625  202.40625 ]
  [101.515625 140.92188  213.21875 ]
  [106.40625  146.40625  215.70312 ]
  ...
  [ 99.671875  52.03125    3.515625]
  [106.109375  56.515625   0.      ]
  [179.0625    68.125      0.890625]]

 [[ 95.109375 134.4375   209.32812 ]
  [100.78125  143.       214.89062 ]
  [107.890625 147.89062  216.89062 ]
  ...
  [ 90.578125  57.90625    8.328125]
  [ 98.46875   87.984375  11.84375 ]
  [240.82812  249.84375  250.07812 ]]

 [[ 98.       142.       213.      ]
  [ 98.       142.       213.      ]
  [106.484375 147.51562  219.51562 ]
  ...
  [ 85.90625   55.875     10.84375 ]
  [ 98.703125  65.        14.0625  ]
  [ 97.1875    53.15625    0.515625]]

 ...

 [[ 54.421875  76.390625  35.9375  ]
  [ 16.9375    37.421875   0.484375]
  [ 18.546875  39.515625   4.03125 ]
  ...
  [114.875    117.0625    21.40625 ]
  [ 39.3125    48.984375  32.40625 ]
  [ 73.09375   70.15625   20.9375  ]]

 [[ 72.453125  87.453125  48.453125]
  [ 32.21875   42.21875 

In [120]:
train_data = train_data.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
validation_data_data = validation_data.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

In [121]:
import sys
import os

# Get the path to the current notebook's directory (D:\Assigments)
notebook_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Construct the full path to the utils_helper directory (D:\theory\utils_helper)
# Go up one level from 'Assigments' to 'D:', then into 'theory/utils_helper'
helper_path = os.path.join(notebook_dir, 'theory', 'utils_helper')

# Check if the path is already in the system path to prevent duplicates
if helper_path not in sys.path:
    # Insert the path at the beginning of the list
    sys.path.insert(0, helper_path)

print(f"Added to system path: {helper_path}")

Added to system path: d:\DAIAugust2025\Python\Anurag\DeepLearning\theory\utils_helper


## Model Building

In [192]:
def build_model(input_shape, num_classes):
    model=tf.keras.Sequential()

    model.add(tf.keras.Input(shape=input_shape))
    

    ##SET-1
    model.add(tf.keras.layers.Conv2D(32,(3,3),padding='same'))#(188x188x32)
    model.add(tf.keras.layers.ReLU())
    model.add(tf.keras.layers.MaxPool2D())#(94x94x32)

    ##SET-2
    model.add(tf.keras.layers.Conv2D(64,(3,3)))#(92x92x64)
    model.add(tf.keras.layers.ReLU())
    model.add(tf.keras.layers.MaxPool2D())#(46x46x64)

    ##SET-3
    model.add(tf.keras.layers.Conv2D(128,(3,3)))#(44x44x128)
    model.add(tf.keras.layers.ReLU())
    model.add(tf.keras.layers.MaxPool2D())#(22x22x128)

    ##SET-4
    model.add(tf.keras.layers.Conv2D(256,(3,3)))#(20x20x256)
    model.add(tf.keras.layers.ReLU())
    model.add(tf.keras.layers.MaxPool2D())#(10x10x256)

    ##SET-5
    model.add(tf.keras.layers.Conv2D(512,(3,3)))#(8x8x512)
    model.add(tf.keras.layers.ReLU())
    model.add(tf.keras.layers.MaxPool2D())#(4x4x512)

    ##SET-6
    model.add(tf.keras.layers.Conv2D(1024,(3,3)))#(2x2x1024)
    model.add(tf.keras.layers.ReLU())

    model.add(tf.keras.layers.Flatten())#4096

    ##HEAD
    model.add(tf.keras.layers.Dense(1024))#1024
    model.add(tf.keras.layers.ReLU())

    model.add(tf.keras.layers.Dense(num_classes))

    return model

In [149]:
IMAGE_HEIGHT,IMAGE_WIDTH = 190,190
model = build_model([IMAGE_HEIGHT,IMAGE_WIDTH,3],5)

In [150]:
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling_4 (Rescaling)         │ (None, 190, 190, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (None, 190, 190, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_21 (ReLU)                 │ (None, 190, 190, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_15 (MaxPooling2D) │ (None, 95, 95, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_20 (Conv2D)              │ (None, 93, 93, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_22 (ReLU)                 │ (None, 93, 93, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_16 (MaxPooling2D) │ (None, 46, 46, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_21 (Conv2D)              │ (None, 44, 44, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_23 (ReLU)                 │ (None, 44, 44, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_17 (MaxPooling2D) │ (None, 22, 22, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_22 (Conv2D)              │ (None, 20, 20, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_24 (ReLU)                 │ (None, 20, 20, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_18 (MaxPooling2D) │ (None, 10, 10, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_23 (Conv2D)              │ (None, 8, 8, 512)      │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_25 (ReLU)                 │ (None, 8, 8, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_19 (MaxPooling2D) │ (None, 4, 4, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_24 (Conv2D)              │ (None, 2, 2, 1024)     │     4,719,616 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_26 (ReLU)                 │ (None, 2, 2, 1024)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 1024)           │     4,195,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_27 (ReLU)                 │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 5)              │         5,125 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,488,645 (40.01 MB)

 Trainable params: 10,488,645 (40.01 MB)

 Non-trainable params: 0 (0.00 B)

In [151]:
lr_callback = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=LR_FACTOR,
    patience=LR_PATIENCE,
    verbose=1,
    mode="auto",
    min_delta=0.0001,
    cooldown=0,
    min_lr=MIN_LR,
)

es_callback = tf.keras.callbacks.EarlyStopping(

    monitor="val_loss",
    min_delta=0,
    patience=PATENCE,
    verbose=1,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
    start_from_epoch=0,
)

In [152]:
model.compile(optimizer=tf.keras.optimizers.Adam(ALPHA),loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])

In [153]:
history = model.fit(train_data,validation_data=validation_data,epochs=EPOCHS,callbacks=[es_callback,lr_callback])

Epoch 1/100


ValueError: Exception encountered when calling Conv2D.call().

[1mNegative dimension size caused by subtracting 3 from 2 for '{{node sequential_4_1/conv2d_23_1/convolution}} = Conv2D[T=DT_FLOAT, data_format="NHWC", dilations=[1, 1, 1, 1], explicit_paddings=[], padding="VALID", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true](sequential_4_1/max_pooling2d_18_1/MaxPool2d, sequential_4_1/conv2d_23_1/convolution/ReadVariableOp)' with input shapes: [?,2,2,256], [3,3,256,512].[0m

Arguments received by Conv2D.call():
  • inputs=tf.Tensor(shape=(None, 2, 2, 256), dtype=float32)

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 64, 64, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [130]:
model.fit?

Signature:
model.fit(
    x=None,
    y=None,
    batch_size=None,
    epochs=1,
    verbose='auto',
    callbacks=None,
    validation_split=0.0,
    validation_data=None,
    shuffle=True,
    class_weight=None,
    sample_weight=None,
    initial_epoch=0,
    steps_per_epoch=None,
    validation_steps=None,
    validation_batch_size=None,
    validation_freq=1,
)
Docstring:
Trains the model for a fixed number of epochs (dataset iterations).

Args:
    x: Input data. It can be:
        - A NumPy array (or array-like), or a list of arrays
        (in case the model has multiple inputs).
        - A backend-native tensor, or a list of tensors
        (in case the model has multiple inputs).
        - A dict mapping input names to the corresponding array/tensors,
        if the model has named inputs.
        - A `keras.utils.PyDataset` returning `(inputs, targets)` or
        `(inputs, targets, sample_weights)`.
        - A `tf.data.Dataset` yielding `(inputs, targets)` or
        `(in

## New model with data augmentation 

In [181]:
data_dir = r"D:\dnn_input\flower_photos"
# 2. Define parameters for image loading
IMAGE_SIZE = (190, 190) # Standardize image dimensions
BATCH_SIZE = 32
VALIDATION_SPLIT = 0.2 # 20% of the data will be used for validation

train_datagen = ImageDataGenerator(
    rescale=1./255, # Normalize pixel values to [0, 1]
    rotation_range=20, # Rotate images randomly by up to 20 degrees
    width_shift_range=0.2, # Shift images horizontally by up to 20%
    height_shift_range=0.2, # Shift images vertically by up to 20%
    shear_range=0.2, # Apply shear transformation
    zoom_range=0.2, # Zoom in on images
    horizontal_flip=True, # Flip images horizontally
    fill_mode='nearest', # Strategy for filling in new pixels created by rotation/shift
    validation_split=VALIDATION_SPLIT # THIS IS KEY for the split
)

# Load the training images from the directory
train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical', # Use 'categorical' for multi-class classification
    subset='training', # Specify this is the training set
    seed=42 # Set a seed for reproducibility
)


validation_datagen = ImageDataGenerator(
    rescale=1./255, # Normalize pixel values
    validation_split=VALIDATION_SPLIT # THIS IS KEY for the split
)

# Load the validation images from the directory
validation_generator = validation_datagen.flow_from_directory(
    data_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation', # Specify this is the validation set
    seed=42 # Use the same seed for consistent splitting
)

Found 2939 images belonging to 5 classes.
Found 731 images belonging to 5 classes.


In [ ]:
def build_model(input_shape, num_classes):
    model=tf.keras.Sequential()

    model.add(tf.keras.Input(shape=input_shape))
    model.add(tf.keras.layers.Rescaling(1./255))
    

    ##SET-1
    model.add(tf.keras.layers.Conv2D(32,(3,3),padding='same'))#(188x188x32)
    model.add(tf.keras.layers.ReLU())
    model.add(tf.keras.layers.MaxPool2D())#(94x94x32)

    ##SET-2
    model.add(tf.keras.layers.Conv2D(64,(3,3)))#(92x92x64)
    model.add(tf.keras.layers.ReLU())
    model.add(tf.keras.layers.MaxPool2D())#(46x46x64)

    ##SET-3
    model.add(tf.keras.layers.Conv2D(128,(3,3)))#(44x44x128)
    model.add(tf.keras.layers.ReLU())
    model.add(tf.keras.layers.MaxPool2D())#(22x22x128)

    ##SET-4
    model.add(tf.keras.layers.Conv2D(256,(3,3)))#(20x20x256)
    model.add(tf.keras.layers.ReLU())
    model.add(tf.keras.layers.MaxPool2D())#(10x10x256)

    ##SET-5
    model.add(tf.keras.layers.Conv2D(512,(3,3)))#(8x8x512)
    model.add(tf.keras.layers.ReLU())
    model.add(tf.keras.layers.MaxPool2D())#(4x4x512)

    ##SET-6
    model.add(tf.keras.layers.Conv2D(1024,(3,3)))#(2x2x1024)
    model.add(tf.keras.layers.ReLU())

    model.add(tf.keras.layers.Flatten())#4096

    ##HEAD
    model.add(tf.keras.layers.Dense(1024))#1024
    model.add(tf.keras.layers.ReLU())

    model.add(tf.keras.layers.Dense(num_classes))

    return model

In [190]:
IMAGE_HEIGHT =190
IMAGE_WIDTH =190
build_model([IMAGE_HEIGHT,IMAGE_WIDTH,3],train_generator.num_classes)
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling_4 (Rescaling)         │ (None, 190, 190, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (None, 190, 190, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_21 (ReLU)                 │ (None, 190, 190, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_15 (MaxPooling2D) │ (None, 95, 95, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_20 (Conv2D)              │ (None, 93, 93, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_22 (ReLU)                 │ (None, 93, 93, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_16 (MaxPooling2D) │ (None, 46, 46, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_21 (Conv2D)              │ (None, 44, 44, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_23 (ReLU)                 │ (None, 44, 44, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_17 (MaxPooling2D) │ (None, 22, 22, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_22 (Conv2D)              │ (None, 20, 20, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_24 (ReLU)                 │ (None, 20, 20, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_18 (MaxPooling2D) │ (None, 10, 10, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_23 (Conv2D)              │ (None, 8, 8, 512)      │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_25 (ReLU)                 │ (None, 8, 8, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_19 (MaxPooling2D) │ (None, 4, 4, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_24 (Conv2D)              │ (None, 2, 2, 1024)     │     4,719,616 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_26 (ReLU)                 │ (None, 2, 2, 1024)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 1024)           │     4,195,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_27 (ReLU)                 │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 5)              │         5,125 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 31,465,937 (120.03 MB)

 Trainable params: 10,488,645 (40.01 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 20,977,292 (80.02 MB)

In [191]:
model.compile(optimizer=tf.keras.optimizers.Adam(ALPHA),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history= model.fit(train_generator,
                   validation_data=validation_generator,
                   epochs=EPOCHS,callbacks=[es_callback,lr_callback])

Epoch 1/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 75s 783ms/step - accuracy: 0.2446 - loss: 1.6013 - val_accuracy: 0.2449 - val_loss: 1.6003 - learning_rate: 0.0010
Epoch 2/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 72s 781ms/step - accuracy: 0.2446 - loss: 1.6008 - val_accuracy: 0.2449 - val_loss: 1.6004 - learning_rate: 0.0010
Epoch 3/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 71s 770ms/step - accuracy: 0.2446 - loss: 1.6008 - val_accuracy: 0.2449 - val_loss: 1.6002 - learning_rate: 0.0010
Epoch 4/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 70s 763ms/step - accuracy: 0.2446 - loss: 1.6007 - val_accuracy: 0.2449 - val_loss: 1.6001 - learning_rate: 0.0010
Epoch 5/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 725ms/step - accuracy: 0.2341 - loss: 1.6009
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
92/92 ━━━━━━━━━━━━━━━━━━━━ 71s 771ms/step - accuracy: 0.2446 - loss: 1.6008 - val_accuracy: 0.2449 - val_loss: 1.6002 - learning_rate: 0.0010
Epoch 6/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 70s 763ms/step - accuracy: 0.2446 - los

KeyboardInterrupt: 

In [185]:
train_generator.num_classes

5